In [34]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
import datetime
%matplotlib inline

In [35]:
train_dir='D:/UGP II ideas/project/Facial Expression Recognition/Data Set/train'
test_dir='D:/UGP II ideas/project/Facial Expression Recognition/Data Set/test'
print(len(os.listdir(train_dir)))
print(len(os.listdir(test_dir)))

7
7


In [36]:
img=cv2.imread('D:/UGP II ideas/project/Facial Expression Recognition/Data Set/train/angry/Training_10290703.jpg')
# img_binary = cv2.
# ret,thresh = cv2.threshold(img,90,120,0)
#plt.imshow(thresh)
img.shape

(48, 48, 3)

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True,
    rescale=1.0/255.0, 
    width_shift_range=0.1,
    height_shift_range=0.1)
val_datagen=ImageDataGenerator(rescale=1.0/255.0,validation_split=0.2)

trainDatagen=train_datagen.flow_from_directory(train_dir,
                                 target_size=(48,48),
                                 batch_size=64,
                                 class_mode='categorical',
                                 color_mode='grayscale',subset='training')
                                
valDatagen=val_datagen.flow_from_directory(test_dir,
                                      target_size=(48,48),
                                      batch_size=64,
                                      class_mode='categorical',
                                      color_mode='grayscale',subset='validation')


Found 28709 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


In [83]:
from tensorflow.keras import regularizers

In [18]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Convolution2D(filters=32,kernel_size=3 ,activation='relu', input_shape=[48, 48, 1]))
cnn.add(tf.keras.layers.Convolution2D(filters=64,kernel_size=3 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Convolution2D(filters=128,kernel_size=5 ,activation='relu',padding='same'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Convolution2D(filters=512,kernel_size=3, padding='same', activation='relu',kernel_regularizer=regularizers.l2(0.01)))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Convolution2D(filters=512,kernel_size=3, padding='same', activation='relu',kernel_regularizer=regularizers.l2(0.01)))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Flatten()) 
cnn.add(tf.keras.layers.Dense(256,activation = 'relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.25))
    
cnn.add(tf.keras.layers.Dense(512,activation = 'relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Dense(7, activation='softmax'))
cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 conv2d_16 (Conv2D)          (None, 46, 46, 64)        18496     
                                                                 
 batch_normalization_18 (Bat  (None, 46, 46, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 dropout_18 (Dropout)        (None, 23, 23, 64)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 23, 23, 128)      

In [19]:
class mycallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs.get('accuracy')>=0.99):
            self.model.stop_training=True
            print("MODEL AS REACHED AMAZING ACCURACY")

In [20]:

cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
history=cnn.fit(x=trainDatagen,epochs=60,validation_data=valDatagen,callbacks=[callbacks],shuffle=True)

Epoch 1/60
  3/449 [..............................] - ETA: 10:52 - loss: 9.9795 - accuracy: 0.1250 

KeyboardInterrupt: 

In [89]:
cnn.save('model1_aug.h5')

In [30]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
def l2_multiclass_svm_loss(y_true, y_pred):
    """
    L2 SVM loss function for multiclass classification
    y_true: one-hot encoded true labels, shape=(batch_size, num_classes)
    y_pred: predicted scores, shape=(batch_size, num_classes)
    """
    # Set margin
    margin = 1.0
    
    # Get number of classes
    num_classes = K.int_shape(y_pred)[-1]
    
    # Get true scores
    true_scores = tf.reduce_sum(y_true * y_pred, axis=-1)
    
    # Compute maximum margins
    max_margins = K.maximum(0.0, y_pred - tf.expand_dims(true_scores, axis=-1) + margin)
    max_margins = max_margins - y_true * max_margins
    
    # Compute loss
    loss = K.sum(K.square(max_margins)) / 2.0
    
    return loss

model = Sequential()

model.add(Convolution2D(filters=32, kernel_size=3, activation='relu', input_shape=[48, 48, 1]))
model.add(Convolution2D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.25))

model.add(Convolution2D(filters=128, kernel_size=5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.25))

model.add(Convolution2D(filters=512, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.25))

model.add(Convolution2D(filters=512, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.25))

model.add(Flatten()) 
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(7, activation='linear'))
model.compile(loss=l2_multiclass_svm_loss, optimizer=Adam(), metrics=['accuracy'])

model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 conv2d_41 (Conv2D)          (None, 46, 46, 64)        18496     
                                                                 
 batch_normalization_48 (Bat  (None, 46, 46, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 dropout_48 (Dropout)        (None, 23, 23, 64)        0         
                                                                 
 conv2d_42 (Conv2D)          (None, 23, 23, 128)      

In [31]:
history1 = model.fit(trainDatagen,
                               epochs=60,
                               validation_data=valDatagen,
                               shuffle=True)

Epoch 1/60
449/449 [==============================] - 729s 2s/step - loss: 371.3353 - accuracy: 0.1950 - val_loss: 187.0491 - val_accuracy: 0.2458
Epoch 2/60
449/449 [==============================] - 466s 1s/step - loss: 229.8450 - accuracy: 0.2180 - val_loss: 193.8729 - val_accuracy: 0.2388
Epoch 3/60
449/449 [==============================] - 361s 804ms/step - loss: 188.3392 - accuracy: 0.2601 - val_loss: 312.3157 - val_accuracy: 0.2598
Epoch 4/60
449/449 [==============================] - 378s 843ms/step - loss: 172.3914 - accuracy: 0.3038 - val_loss: 202.0408 - val_accuracy: 0.3024
Epoch 5/60
449/449 [==============================] - 386s 860ms/step - loss: 162.7188 - accuracy: 0.3397 - val_loss: 176.9366 - val_accuracy: 0.2849
Epoch 6/60
449/449 [==============================] - 362s 806ms/step - loss: 145.4058 - accuracy: 0.3983 - val_loss: 130.6048 - val_accuracy: 0.4309
Epoch 7/60
449/449 [==============================] - 285s 635ms/step - loss: 130.5344 - accuracy: 0.4388 

449/449 [==============================] - 308s 685ms/step - loss: 96.5510 - accuracy: 0.6135 - val_loss: 104.8827 - val_accuracy: 0.6215
Epoch 56/60
449/449 [==============================] - 292s 651ms/step - loss: 98.3148 - accuracy: 0.6091 - val_loss: 92.5590 - val_accuracy: 0.6425
Epoch 57/60
449/449 [==============================] - 289s 644ms/step - loss: 97.4639 - accuracy: 0.6133 - val_loss: 88.4247 - val_accuracy: 0.6536
Epoch 58/60
449/449 [==============================] - 290s 645ms/step - loss: 96.2439 - accuracy: 0.6162 - val_loss: 92.6267 - val_accuracy: 0.6292
Epoch 59/60
449/449 [==============================] - 291s 648ms/step - loss: 96.2385 - accuracy: 0.6127 - val_loss: 90.0969 - val_accuracy: 0.6564
Epoch 60/60
449/449 [==============================] - 313s 697ms/step - loss: 96.2346 - accuracy: 0.6137 - val_loss: 89.5117 - val_accuracy: 0.6306


In [32]:
model.save('model_optimal_svm_loss.h5')